#  AI Search - Search index

### Load .env file (Copy .env-sample to .env and update accordingly)

In [1]:
import os
from dotenv import load_dotenv

load_dotenv(override=True) # take environment variables from .env.

from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential

endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
credential = AzureKeyCredential(os.environ["AZURE_SEARCH_ADMIN_KEY"]) if len(os.environ["AZURE_SEARCH_ADMIN_KEY"]) > 0 else DefaultAzureCredential()
index_name = os.environ["AZURE_SEARCH_INDEX"]

### Dependencies

In [2]:
#! pip install -r requirements.txt

### Global variables

In [3]:
from azure.search.documents import SearchClient

search_client = SearchClient(endpoint, index_name, credential=credential)

### Global evaluation helper

In [13]:
def evaluate_results(results):
    i=0 
    for result in results:  
        i=i+1
        print(f"Result index: {i}, score: {result['@search.score']}, title: {result['title']}") # , content: {result['content']}
        if '@search.reranker_score' in result and result['@search.reranker_score'] is not None:   
            print(f"Semantic reranker score: {result['@search.reranker_score']}")

### Query text to be used in all queries

In [14]:
query = "information theory"  
# query = "ZYX1"

### Perform a vector similarity search
This example shows a pure vector search using the vectorizable text query, all you need to do is pass in text and your vectorizer will handle the query vectorization.

In [15]:
from azure.search.documents.models import VectorizableTextQuery

# https://learn.microsoft.com/en-us/python/api/azure-search-documents/azure.search.documents.models.vectorizabletextquery?view=azure-python-preview
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=5, fields="contentVector", exhaustive=True)
# Use the below query to pass in the raw vector query instead of the query vectorization
# vector_query = RawVectorQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="vector")
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["title", "chunk_id", "content"],
    # filter="type eq 1",
    top=5
)  
  
evaluate_results(results)

Result index: 1, score: 0.8818742, title: 266 # Christoph Adami on How Information Makes Sense of Biology.txt
Result index: 2, score: 0.8798266, title: 266 # Christoph Adami on How Information Makes Sense of Biology.txt
Result index: 3, score: 0.87780064, title: 266 # Christoph Adami on How Information Makes Sense of Biology.txt
Result index: 4, score: 0.8686019, title: 266 # Christoph Adami on How Information Makes Sense of Biology.txt
Result index: 5, score: 0.8667384, title: 266 # Christoph Adami on How Information Makes Sense of Biology.txt


## Hybrid search using vectors and full text in Azure AI Search

https://learn.microsoft.com/en-us/azure/search/hybrid-search-overview

Hybrid search is a combination of full text and vector queries that execute against a search index that contains both searchable plain text content and generated embeddings. For query purposes, hybrid search is:

- A single query request that includes both search and vectors query parameters
- Executing in parallel
- With merged results in the query response, scored using Reciprocal Rank Fusion (RRF, https://learn.microsoft.com/en-us/azure/search/hybrid-search-ranking)


In [16]:
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=5, fields="contentVector", exhaustive=True)
  
results = search_client.search(  
    search_text=query,  
    vector_queries= [vector_query],
    select=["title", "chunk_id", "content"],
    top=5
)  

evaluate_results(results)

Result index: 1, score: 0.032786883413791656, title: 266 # Christoph Adami on How Information Makes Sense of Biology.txt
Result index: 2, score: 0.03159204125404358, title: 266 # Christoph Adami on How Information Makes Sense of Biology.txt
Result index: 3, score: 0.03128054738044739, title: 266 # Christoph Adami on How Information Makes Sense of Biology.txt
Result index: 4, score: 0.03125, title: 266 # Christoph Adami on How Information Makes Sense of Biology.txt
Result index: 5, score: 0.029571646824479103, title: 266 # Christoph Adami on How Information Makes Sense of Biology.txt


## Semantic ranking in Azure AI Search

https://learn.microsoft.com/en-us/azure/search/semantic-search-overview?source=recommendations

In [17]:
from azure.search.documents.models import (
    QueryType,
    QueryCaptionType,
    QueryAnswerType
)

vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=5, fields="contentVector", exhaustive=True)

results = search_client.search(  
    search_text=query,
    vector_queries=[vector_query],
    select=["title", "chunk_id", "content"],
    query_type=QueryType.SEMANTIC,
    semantic_configuration_name='my-semantic-config',
    query_caption=QueryCaptionType.EXTRACTIVE,
    query_answer=QueryAnswerType.EXTRACTIVE,
    top=4
)

semantic_answers = results.get_answers()
if semantic_answers:
    for answer in semantic_answers:
        if answer.highlights:
            print(f"Semantic Answer: {answer.highlights}")
        else:
            print(f"Semantic Answer: {answer.text}")
        print(f"Semantic Answer Score: {answer.score}\n")


evaluate_results(results)

Result index: 1, score: 0.03128054738044739, title: 266 # Christoph Adami on How Information Makes Sense of Biology.txt
Semantic reranker score: 2.989736557006836
Result index: 2, score: 0.014285714365541935, title: 266 # Christoph Adami on How Information Makes Sense of Biology.txt
Semantic reranker score: 2.5531556606292725
Result index: 3, score: 0.032786883413791656, title: 266 # Christoph Adami on How Information Makes Sense of Biology.txt
Semantic reranker score: 2.546262264251709
Result index: 4, score: 0.014705882407724857, title: 266 # Christoph Adami on How Information Makes Sense of Biology.txt
Semantic reranker score: 2.538602828979492
